# Developer Insights: use typing lib

we need type conversions from omnikeeper attribute types used in graphql to python native types and back


In [1]:
import omnikeeper_client as ok
import omnikeeper_client.typing as okc_typing

import uuid


## omnikeeper to python

In [2]:
omninkeeper_response_mergedattributes = [
        {
          "attribute": {
            "name": "__name",
            "value": {
              "type": "TEXT",
              "isArray": False,
              "values": [
                "test-ci01"
              ]
            }
          }
        },
        {
          "attribute": {
            "name": "stringarray",
            "value": {
              "type": "TEXT",
              "isArray": True,
              "values": [
                "test_value_1",
                "test_value_2",
              ]
            }
          }
        },
        {
          "attribute": {
            "name": "integerarray",
            "value": {
              "type": "INTEGER",
              "isArray": True,
              "values": [
                1,
                2,
                3,
              ]
            }
          }
        },
      ]

now convert this repsonse to a dict of legacy types, using attribute name as key of dict

In [3]:
okc_typing.attributes_to_dict(omninkeeper_response_mergedattributes)

{'__name': 'test-ci01',
 'stringarray': ['test_value_1', 'test_value_2'],
 'integerarray': [1, 2, 3]}

*Details:* ``attributes_to_dict`` uses ``attributevalue_to_pythonvalue`` to convert every single attribute, to test that we can use:

In [4]:
attribute_name = omninkeeper_response_mergedattributes[0]['attribute']['name']
attribute_value = okc_typing.attributevalue_to_pythonvalue(omninkeeper_response_mergedattributes[0]['attribute']['value'])

print (f"{attribute_name} is of type {type(attribute_value)} with value {attribute_value}")

__name is of type <class 'str'> with value test-ci01


## python native types to omnikeeper attribute types

### detecting types

to give convenience to users we have to detect types of a single value (non array!)

In [5]:
okc_typing._detect_single_attributetype(1)

'INTEGER'

In [75]:
okc_typing._detect_single_attributetype(3.1415)

'DOUBLE'

In [76]:
okc_typing._detect_single_attributetype(True)

'BOOLEAN'

In [77]:
okc_typing._detect_single_attributetype("Hello World!")

'TEXT'

In [78]:
okc_typing._detect_single_attributetype("Hello\nWorld!")

'MULTILINE_TEXT'

In [79]:
okc_typing._detect_single_attributetype({'foo': 'bar'})

'JSON'

In [80]:
okc_typing._detect_single_attributetype([1,2,3,4])

'JSON'

Demonstrate how to convert a dict to an omnikeeper attribute list using autodetect and typ hints to overrule

In [6]:
my_demo_ciid = uuid.uuid1()

python_native_attributes = {
    "author": "Markus Hof",
    "description": "This is\nvery\nconvenient!",
    "counter": [5,2,3,4],
    "lines": ["lorem", "ipsum"],
    "complex": {"foo": "bar", "counter": 123, "subobject": {"yellow": "submarine"}},
    "is_this_good": True,
    "it_could_be_a_integer": 42,
    "unknown_type_list": [],
    "hinted_empty_list": [],
}

okc_typing.dict_to_attributes(my_demo_ciid, python_native_attributes, type_hints={
    'it_could_be_a_integer': ok.ATTRIBUTETYPE_TEXT,
    'hinted_empty_list': ok.ATTRIBUTETYPE_INTEGER,
    })

[{'ci': '08cbccbc-c4cf-11ee-a5e0-00155ded4619',
  'name': 'author',
  'value': {'type': 'TEXT', 'isArray': False, 'values': ['Markus Hof']}},
 {'ci': '08cbccbc-c4cf-11ee-a5e0-00155ded4619',
  'name': 'description',
  'value': {'type': 'MULTILINE_TEXT',
   'isArray': False,
   'values': ['This is\nvery\nconvenient!']}},
 {'ci': '08cbccbc-c4cf-11ee-a5e0-00155ded4619',
  'name': 'counter',
  'value': {'type': 'INTEGER',
   'isArray': True,
   'values': ['5', '2', '3', '4']}},
 {'ci': '08cbccbc-c4cf-11ee-a5e0-00155ded4619',
  'name': 'lines',
  'value': {'type': 'TEXT', 'isArray': True, 'values': ['lorem', 'ipsum']}},
 {'ci': '08cbccbc-c4cf-11ee-a5e0-00155ded4619',
  'name': 'complex',
  'value': {'type': 'JSON',
   'isArray': False,
   'values': ['{"foo": "bar", "counter": 123, "subobject": {"yellow": "submarine"}}']}},
 {'ci': '08cbccbc-c4cf-11ee-a5e0-00155ded4619',
  'name': 'is_this_good',
  'value': {'type': 'BOOLEAN', 'isArray': False, 'values': [True]}},
 {'ci': '08cbccbc-c4cf-11ee-